# Use Wikipedia as alternative data source

In [214]:
import numpy as np
import pandas as pd
import wikipedia
import re
import timeit
from bs4 import BeautifulSoup
import time
import bs4

## Load Formated Data

In [16]:
country_df = pd.read_pickle("../../data/Tables/country_df.pkl")
comes_from_df = pd.read_pickle("../../data/Tables/comes_from_df.pkl")
genre_df = pd.read_pickle("../../data/Tables/genre_df.pkl")
is_of_type_df = pd.read_pickle("../../data/Tables/is_of_type_df.pkl")
language_df = pd.read_pickle("../../data/Tables/language_df.pkl")
spoken_languages_df = pd.read_pickle("../../data/Tables/spoken_languages_df.pkl")
character_df = pd.read_pickle("../../data/Tables/character_df.pkl")
actor_df = pd.read_pickle("../../data/Tables/actor_df.pkl")
movie_df = pd.read_pickle("../../data/Tables/movie_df.pkl")
belongs_to_df = pd.read_pickle("../../data/Tables/belongs_to_df.pkl")
play_df = pd.read_pickle("../../data/Tables/play_df.pkl")
appears_in_df = pd.read_pickle("../../data/Tables/appears_in_df.pkl")

## Parser

In [219]:
default_movie_entries = set(["Directed by","Written by","Produced by","Starring",
                            "Cinematography","Edited by","Music by","Distributed by",
                            "Release date","Running time","Country","Language",
                            "Budget","Box office","Screenplay by","Based on",
                            "Release dates","Story by","Director of Animation"])
def get_movie_data(page_id,entry_keys=default_movie_entries):
    page = wikipedia.page(pageid=page_id)
    page_parser = BeautifulSoup(page.html(),"html.parser")
    table_data = page_parser.find_all("table",class_="infobox")[0]
    table_data_list = [s for s in list(table_data.descendants)[0].strings 
                       if re.match("\A[(,),\n,\[,\]]",s) == None]
    entry_indices = [idx for (idx,entry) in enumerate(table_data_list)
                    if entry in entry_keys]
    entry_indices.append(len(table_data_list))
    table_data_dict = dict([(table_data_list[entry_indices[i]],
                             table_data_list[entry_indices[i]+1:entry_indices[i+1]])
                           for i in range(len(entry_indices)-1)])
    return table_data_dict,page.summary

In [220]:
a = get_movie_data(12305190)

In [221]:
a

({'Directed by': ['Norodom Sihanouk'],
  'Written by': ['Norodom Sihanouk'],
  'Produced by': ['Norodom Sihanouk'],
  'Starring': ['Norodom Sihamoni', 'San Chariya'],
  'Release date': ['1992'],
  'Running time': ['63 minutes'],
  'Country': ['Cambodia'],
  'Language': ['Khmer']},
 'My Village at Sunset  is a 1992 Cambodian romantic drama film directed by Norodom Sihanouk.')

### Tests

In [208]:
movie_df[movie_df["release_date"].isna()].sample(10)

,name,release_date,revenue,runtime,freebase_id,plot
movie_id,,,,,,
28340819,Montana Fishburne: An A-List Daughter Makes he...,NaN,NaN,65.0,/m/0cny8lr,NaN
12305190,My Village at Sunset,NaN,NaN,63.0,/m/02vzp06,"After the 1991 peace agreements, a handsome, ..."
6399567,The little cars in the great race,NaN,NaN,NaN,/m/0g3z6w,NaN
24364162,Nidhiyude Katha,NaN,NaN,75.0,/m/07s81gr,NaN
32865605,Scumbus,NaN,NaN,NaN,/m/0h3rnv0,*Ed Kavalee as Tom *Toby Truslove as Red *Lac...
7299021,The Unseeable,NaN,NaN,100.0,/m/0kv4b0,"Set in 1934 Siam, the story involves a young p..."
25067458,Orgasmic Birth: The Best-Kept Secret,NaN,NaN,85.0,/m/09gjlx8,NaN
20272899,Mary from Beijing,NaN,NaN,NaN,/m/04zx5sr,NaN
19286251,Maadappura,NaN,NaN,NaN,/m/04lf78v,NaN
